#Vacation Py

In [1]:
# Setup and Importing Dependencies

import gmaps
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import requests

In [2]:
# Imports and defines API Key

from api_keys import g_key

##Store WeatherPy results into DataFrame
* Loads the .csv exported in WeatherPy to a new DataFrame

In [3]:
vacation_py_df = pd.read_csv("../WeatherPy/Output_Data/City_Weather.csv")

# Prints DataFrame
vacation_py_df

,Unnamed: 0,City,Temperature,Humidity,Longitude,Latitude,Cloudiness,Country,Windspeed,Date
0,0,lugovoy,44.40,55,72.76,42.95,29,KZ,5.61,1603667993
1,1,yulara,66.20,30,130.99,-25.24,28,AU,20.80,1603668257
2,2,kapaa,82.99,74,-159.32,22.08,40,US,9.17,1603668207
3,3,villa carlos paz,64.99,62,-64.50,-31.42,65,AR,15.26,1603668259
4,4,klaksvik,42.80,87,-6.59,62.23,26,FO,6.93,1603668260
...,...,...,...,...,...,...,...,...,...,...
587,587,manado,82.40,78,124.85,1.49,40,ID,1.74,1603668396
588,588,mushie,72.68,95,16.90,-3.02,98,CD,1.83,1603668521
589,589,belaya gora,9.99,93,146.42,68.53,10,RU,6.73,1603668522
590,590,togur,26.76,98,82.83,58.36,99,RU,10.18,1603668523


##Humidity Heatmap
* Configures gmaps
* Uses the Latitude and Longitude as locations as Humidity as the weight.
* Adds Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [7]:
# Variables
Locations = vacation_py_df[["Latitude", "Longitude"]]
Humidity = vacation_py_df["Humidity"]
figure = gmaps.figure()

# Creates Heat Layer
Heatmap_Layer = gmaps.heatmap_layer(Locations, weights=Humidity,
                                   dissipating=False, max_intensity=10,
                                   point_radius=1)

# Adds Heatmap Layer and Displays Figure
figure.add_layer(Heatmap_Layer)
figure

Figure(layout=FigureLayout(height='420px'))

##Making a New DataFrame to fit my Weather Criteria
Narrows down the cities to fit the following weather conditions and drops any rows with null values:
* A max temperature lower than 70 but higher than 60
* Wind speed less than 5 mph
* Cloudiness less than 5.0

##Hotel Map
* Stores into variable named `hotel_df`.
* Adds a "Hotel Name" column to the DataFrame
* Sets the parameters to search for hotels with 5000 meters.
* Hits the Google Places API for each city's coordinates.
* Stores the first Hotel result into the DataFrame.
* Plots markers on top of the heatmap.